In [1]:
import os
import glob
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
import pickle
import shutil
import run_birdnet as rb
import splicer as sp

# Load and initialize the BirdNET-Analyzer models.
# analyzer = Analyzer()


def make_dir(new_dir):
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
    return "made dir " + str(new_dir)

def mass_run_all_dates(code, datasheet_per_date_birdnet_only, per_date_birdnet_only, conf):
    #given
    os.chdir(datasheet_per_date_birdnet_only)
    file_list= glob.glob("*csv")
    dates= [i[0:8] for i in file_list]
    dates=dates[:-1]

    super_stats_df=pd.DataFrame(columns=["date", "# birdnet x datasheet", "time code" ])

    for iterant in range(len(dates)):
        date= dates[iterant]

        os.chdir(datasheet_per_date_birdnet_only)
        datasheet_split = glob.glob(date+"*")
        print(datasheet_split)
        df_ds_split= pd.read_csv(datasheet_split[0])

        os.chdir(per_date_birdnet_only)
        birdnet_split = glob.glob(date+"*")
        df_bd_split= pd.read_csv(birdnet_split[0])

        array_times= df_ds_split['time code']
        array_times=[str(i) for i in array_times]
        array_times=["0"+i if len(i)<2 else i for i in array_times]
        df_ds_split['time code']= array_times
        df_ds_split
        array_times= df_bd_split['time']
        array_times=[str(i) for i in array_times]
        array_times=["0"+i[1] if i[0]=="_" else i for i in array_times]
        df_bd_split['time']= array_times

        df_bd_split["common name"]=rb.remove_special_from_names(df_bd_split["common name"])
        df_bd_split

        date= dates[iterant]

        stats_df= pd.DataFrame(columns=["date","# birdnet x datasheet", "time code"])

        input_time_interval=5

        time_codes= np.arange(0, 60, input_time_interval)

        time_codes=[str(i) for i in time_codes]
        time_codes=["0"+i if len(i)<2 else i for i in time_codes]

        # bd_minus_ds_array=[]

        print(date)
        for it in range(len(time_codes)):
            time= time_codes[it]
            new_df= pd.DataFrame(columns=["date","# birdnet x datasheet", "time code"])
            ds_groupby= df_ds_split.groupby("time code")
            ds_compare= ds_groupby.get_group(time_codes[it])
            ds_compare_array=set(np.array(ds_compare["common name"]))
            

            bd_groupby= df_bd_split.groupby("time")
            try:
                bd_compare= bd_groupby.get_group(time_codes[it])
                bd_compare_array=set(np.array(bd_compare["common name"]))
            except KeyError:
                bd_compare_array=set([])

            bd_intersection_ds= bd_compare_array.copy()
            bd_intersection_ds= bd_intersection_ds.intersection(ds_compare_array)
            new_df["# birdnet x datasheet"]= list(bd_intersection_ds)
            new_df["date"]= date
            new_df["time code"]=time
            stats_df= pd.concat([stats_df, new_df])
        stats_df["date"]= date
        # stats_df.to_csv(date+"_stats_w_names_cl.csv")  
        super_stats_df= pd.concat([super_stats_df, stats_df])

    dir= code+"\\stats_tp\\"
    make_dir(dir)
    os.chdir(dir)
    super_stats_df.to_csv("all_dates_merged_w_names_"+str(conf)+"confidence.csv")
    return "done :D"



# def run_birdnet_and_fetch_files(samples_audio, big_dir, common_resources, conf_threshold_for_bn):
    
#     with open(filename, 'rb') as handle:
#         data = pickle.load(handle)

C:\Users\adity\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.


In [16]:
def get_start_time_info_per_bird_per_timecode(dir, date, time_code, bird):
    os.chdir(dir)
    date_folder= glob.glob(date+"*")[0]
    os.chdir(dir+ date_folder)
    
    file_name= glob.glob("*"+time_code+".pickle")[0]
    with open(file_name, 'rb') as handle:
        data = pickle.load(handle)
    try:
        array_start_time= data[bird]
    except KeyError:
        array_start_time= []
    return array_start_time

def get_piece_of_audio(in_folder, audio_file, start_time, bird, dir_to_store, split):
    # os.chdir(sound_folder)
    os.chdir(split)
    os.chdir(in_folder)
    audio, sr = librosa.load(audio_file)

    end_time= start_time+3
    slice= audio[int(start_time)*sr: int(end_time)*sr]
    out_filename =  audio_file[:-4] +"_"+str(start_time) +"_piece_of_"+str(bird)+ ".WAV"

    os.chdir(dir_to_store)
    sf.write(out_filename, slice, sr)
    print("pieced "+ bird)
    return None

def fetch_files(dir, date, time_code, bird, dir_tp_files, split):
    array_start_time= get_start_time_info_per_bird_per_timecode(dir, date, time_code, bird)
    dir_to_store= dir_tp_files + bird+ "\\"
    make_dir(dir_to_store)
    #find date folder in audio split
    os.chdir(split)
    input_folder_name= glob.glob(date+"*")[0]
    print(input_folder_name)
    audio_file_name= date+"_split_"+time_code+".WAV"
    if len(array_start_time)==0:
        return None
    else:
        for start_time in array_start_time:
            get_piece_of_audio(input_folder_name, audio_file_name, start_time, bird, dir_to_store, split)

In [18]:
# dir= r"D:\Research\Rohini 2024\Bird Data\common_resources\store_birdnet_info\\"
# common_resources= r"D:\Research\Rohini 2024\Bird Data\common_resources\\"
# storage= "d:\\Research\\Rohini 2024\\Bird Data\\" 
# time_interval=5 
# conf=0.3
# new_dir, code, split, datasheet_per_date_birdnet_only, per_date_birdnet_only= sp.create_required_directories(storage, time_interval, conf)
# split= storage+ "audio_files_split\\"
# stats_tp= code+"\\stats_tp\\"

def mass_run_fetch_files(confusion_type, conf, code, split):
    dir_tp_files= code + confusion_type+"_audio_files\\"
    make_dir(dir_tp_files)
    stats= code+"\\stats_"+confusion_type+"\\"
    os.chdir(stats)
    merged_dataframe= pd.read_csv("all_dates_merged_w_names_"+str(conf)+"confidence.csv")
    grouped_df= merged_dataframe.groupby("name")
    unique_bird_names= list(set(list(merged_dataframe["name"])))

    for bird in unique_bird_names:

        bird_dir= dir_tp_files+ bird+ "\\"
        make_dir(bird_dir)
        bird_df= grouped_df.get_group(bird)
        date_list, time_list= list(bird_df["date"]), list(bird_df["time code"])
        date_time_tuple_list= [(date_list[i],time_list[i]) for i in range(len(bird_df["# birdnet x datasheet"]))]

        for tuple in date_time_tuple_list:
            date= str(tuple[0])
            time_code= tuple[1]
            time_code= str(int(time_code))
            fetch_files(dir, date, time_code, bird, dir_tp_files, split)

20230309_075723
pieced greenish warbler
20230309_075723
pieced greenish warbler
pieced greenish warbler
pieced greenish warbler
20230413_065745
pieced greenish warbler
pieced greenish warbler
pieced greenish warbler
pieced greenish warbler
pieced greenish warbler
pieced greenish warbler
pieced greenish warbler
pieced greenish warbler
20230907_070226
pieced greenish warbler
pieced greenish warbler
pieced greenish warbler
20230907_070226
pieced greenish warbler
pieced greenish warbler
20230907_070226
pieced greenish warbler
20230907_070226
pieced greenish warbler
pieced greenish warbler
20230309_075723
pieced oriental magpie robin
pieced oriental magpie robin
20230411_070055
pieced oriental magpie robin
pieced oriental magpie robin
pieced oriental magpie robin
pieced oriental magpie robin
pieced oriental magpie robin
20230411_070055
pieced oriental magpie robin
pieced oriental magpie robin
20230411_070055
pieced oriental magpie robin
20230425_065840
pieced oriental magpie robin
20230425_

In [3]:
fetch_files(dir, date, time_code, bird, dir_tp_files, split)

20230309_075723
pieced purple rumped sunbird
